## Import Libraries

In [1]:
from neo4j import GraphDatabase, basic_auth
from pyvis.network import Network
import os
import webbrowser
import time

## Data Loading

In [2]:
# Load config from file
def load_neo4j_config(filepath):
    config = {}
    with open(filepath, 'r') as file:
        for line in file:
            if '=' in line:
                key, value = line.strip().split('=', 1)
                config[key] = value
    return config

In [3]:
# Load credentials from the file
config = load_neo4j_config("../../Encryption/Blank-Sandbox.txt")

In [4]:
# Assign values
uri = config.get("NEO4J_URI")
username = config.get("NEO4J_USERNAME")
password = config.get("NEO4J_PASSWORD")
database = config.get("NEO4J_DATABASE", "neo4j")

## Customer Relationship Database Class

In [5]:
class customerRelationsDatabase:

    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(
            uri,
            auth=basic_auth(user, password),
        )

    def initialise_graph(self):
        with self.driver.session() as session:
            # Clear existing nodes and relationships
            session.run("MATCH (n) DETACH DELETE n")

            # Create Customer nodes
            session.run("""
                CREATE 
                    (:Customer {name: 'Sophie'}),
                    (:Customer {name: 'Jeremy'}),
                    (:Customer {name: 'Clem'}),
                    (:Customer {name: 'Antonio'}),
                    (:Customer {name: 'Rosa'}),
                    (:Customer {name: 'Karl'}),
                    (:Customer {name: 'Nye'})
            """)

            # Create Product nodes
            session.run("""
                CREATE 
                    (:Product {model: 'E215', mfr: 'Shure', category: 'Audio|Wired|Earphones|Products'}),
                    (:Product {model: 'QC3511', mfr: 'Bose', category: 'Audio|Wired and Wireless|Earphones|Products'}),
                    (:Product {model: 'Earpods', mfr: 'Apple', category: 'Audio|Wireless|Earphones|Products'})
            """)

            # Create BOUGHT relationships
            session.run("""
                MATCH (c:Customer {name: 'Sophie'}), (p:Product {model: 'E215'})
                CREATE (c)-[:BOUGHT]->(p)
            """)
            session.run("""
                MATCH (c:Customer {name: 'Jeremy'}), (p:Product {model: 'E215'})
                CREATE (c)-[:BOUGHT]->(p)
            """)
            session.run("""
                MATCH (c:Customer {name: 'Clem'}), (p:Product {model: 'QC3511'})
                CREATE (c)-[:BOUGHT]->(p)
            """)
            session.run("""
                MATCH (c:Customer {name: 'Antonio'}), (p:Product {model: 'E215'})
                CREATE (c)-[:BOUGHT]->(p)
            """)
            session.run("""
                MATCH (c:Customer {name: 'Rosa'}), (p:Product {model: 'E215'})
                CREATE (c)-[:BOUGHT]->(p)
            """)
            session.run("""
                MATCH (c:Customer {name: 'Karl'}), (p:Product {model: 'E215'})
                CREATE (c)-[:BOUGHT]->(p)
            """)
            session.run("""
                MATCH (c:Customer {name: 'Karl'}), (p:Product {model: 'Earpods'})
                CREATE (c)-[:BOUGHT]->(p)
            """)
            session.run("""
                MATCH (c:Customer {name: 'Nye'}), (p:Product {model: 'Earpods'})
                CREATE (c)-[:BOUGHT]->(p)
            """)

    def visualise_graph(self):
        with self.driver.session() as session:
            query = """
                MATCH (c:Customer)-[:BOUGHT]->(p:Product)
                RETURN c.name AS customer, p.model AS model, p.mfr AS mfr, p.category AS category
            """
            results = session.run(query)

            net = Network(height='750px', width='100%', bgcolor='#ffffff', font_color='black', directed=True)
            net.force_atlas_2based()

            added_nodes = set()
            added_edges = set()  # Track (source, target)

            for record in results:
                cust = record["customer"]
                model = record["model"]
                mfr = record["mfr"]
                category = record["category"]
                product_label = f"{mfr} {model}"

                # Add nodes
                if cust not in added_nodes:
                    net.add_node(cust, label=cust, shape='ellipse', color='lightblue')
                    added_nodes.add(cust)

                if product_label not in added_nodes:
                    net.add_node(product_label, label=f"{product_label}\n{category}", shape='box', color='orange')
                    added_nodes.add(product_label)

                # Add edge only once
                edge_key = (cust, product_label)
                if edge_key not in added_edges:
                    net.add_edge(cust, product_label, label='BOUGHT')
                    added_edges.add(edge_key)

        # Save and open
        path = os.path.abspath("../../Playground/output/customer_product_graph.html")
        net.write_html(path)
        webbrowser.open("file://" + os.path.abspath(path))

    def close(self):
        self.driver.close()

In [6]:
db = customerRelationsDatabase(uri, username, password)
db.initialise_graph()
time.sleep(1)
db.visualise_graph()
db.close()

## Hierarchical Relationships

In [7]:
class Category:
    def __init__(self, name):
        self.name = name
        self.subclasses = []

    def add_subclass(self, subclass):
        self.subclasses.append(subclass)

    def __repr__(self):
        return f"Category(name='{self.name}')"

In [8]:
class Product:
    def __init__(self, model, category):
        self.model = model
        self.category = category
        self.upsell = None

    def set_upsell(self, product):
        self.upsell = product

    def __repr__(self):
        upsell_model = self.upsell.model if self.upsell else "None"
        return (f"Product(model='{self.model}', category='{self.category.name}', "
                f"upsell='{upsell_model}')")

In [9]:
# Create categories
mobile_phone = Category("mobile phone")
ios = Category("iOS")
mobile_phone.add_subclass(ios)  # iOS is a subclass of mobile phone

In [10]:
# Create products and assign to category iOS
iphone_12_mini = Product("iPhone 12 mini", ios)
iphone_12 = Product("iPhone 12", ios)
iphone_12_pro = Product("iPhone 12 Pro", ios)

In [11]:
# Set upsell relationships
iphone_12_mini.set_upsell(iphone_12)
iphone_12.set_upsell(iphone_12_pro)

In [12]:
# Display structure
print("Categories:")
print(f"{mobile_phone.name} has subclasses: {[sub.name for sub in mobile_phone.subclasses]}")

Categories:
mobile phone has subclasses: ['iOS']


In [13]:
print("\nProducts:")
print(iphone_12_mini)
print(iphone_12)
print(iphone_12_pro)


Products:
Product(model='iPhone 12 mini', category='iOS', upsell='iPhone 12')
Product(model='iPhone 12', category='iOS', upsell='iPhone 12 Pro')
Product(model='iPhone 12 Pro', category='iOS', upsell='None')
